<a href="https://colab.research.google.com/github/sAndreotti/MedicalMeadow/blob/main/ATML_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Investigate Dataset

In [10]:
from datasets import load_dataset

ds = load_dataset("medalpaca/medical_meadow_medical_flashcards")
ds = ds['train']
ds

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 33955
})

In [11]:
print(f"Input: {ds['input'][0]}")
print(f"Instruction: {ds['instruction'][0]}")
print(f"Output: {ds['output'][0]}")

Input: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Instruction: Answer this question truthfully
Output: Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.


## Train and evaluate models

## Add voice interactivity

## Potential extensions